In [39]:
import json
from datasets import load_dataset

In [53]:
datasets = load_dataset("hate-speech-portuguese/hate_speech_portuguese", split='train[:10%]')

In [54]:
print(datasets)

Dataset({
    features: ['text', 'label', 'hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'],
    num_rows: 567
})


In [55]:
datasets = datasets.remove_columns(['hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'])
datasets

Dataset({
    features: ['text', 'label'],
    num_rows: 567
})

In [56]:
datasets = datasets.train_test_split(test_size=0.2)

In [58]:
datasets['train']['text'][0]

"@_carmeloneto \nOs 'cumpanhero' quebraram todas as regras."

In [59]:
def removeN(example):
    example["text"] = example["text"].replace("\n", " ")
    return example

In [60]:
datasets = datasets.map(removeN)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [61]:
datasets['train']['text'][0]

"@_carmeloneto  Os 'cumpanhero' quebraram todas as regras."

### label 0 -> No Hate Speech
### label 1 -> Hate Speech

In [64]:
def labelChange(example):
    example["label_text"] = "No Hate Speech" if  example["label"] == 0 else "Hate Speech"
    return example

In [67]:
datasets = datasets.map(labelChange)
datasets

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 114
    })
})

In [68]:
datasets = datasets.remove_columns(['label'])
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label_text'],
        num_rows: 114
    })
})

In [69]:
datasets['train'][0]

{'text': "@_carmeloneto  Os 'cumpanhero' quebraram todas as regras.",
 'label_text': 'No Hate Speech'}